<a href="https://colab.research.google.com/github/serenechack/Remote-Sensing/blob/main/Lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap

In [2]:
# Authenticate, then initialize GEE to import geemap

In [3]:
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bGC2vEA3TLYaTH8yQCfGoIbvcWPLbaq5HjJqEwvSiQ0&tc=5eGow8kJNq4mxEsze-mz-8I0lt0l9_Bk9GSwUJOVQgw&cc=A9TTeMs1R-BJg3sin_89khwt1d5Bn8aSsCyeWbbElGg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnSiaB85Go1q5zhkxaSf7POPyq-eARMyIJn0hUua4CiJrpUy4BJ5Kc

Successfully saved authorization token.


In [4]:
ee.Initialize()
import geemap
geemap.update_package()

Unzipping geemap-master.zip ...
Data downloaded to: /root/Downloads/geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


In [5]:
#m = geemap.Map(center=(40.730610, -73.935242), zoom = 4)

In [6]:
#m = geemap.Map(basemap='HYBRID')
#m.add_basemap('OpenTopoMap')

In [7]:
#url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
#m.add_tile_layer(url, name='Google Satellite', attribution='Google')

In [8]:
#url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
#m.add_wms_layer(url=url, layers='0', name='NAIP Imagery', format='image/png', shown=True)

In [9]:
#image = ee.Image('USGS/SRTMGL1_003')
#vis_params = {
#  'min': 0,
#  'max': 4000,
#  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
#  }
#m.addLayer(image, vis_params, 'SRTM DEM', True, 0.5)

In [10]:
#m.setCenter(-73.935242, 40.730610, zoom=10)

In [11]:
#m.add_legend(builtin_legend='NLCD')

In [12]:
m = geemap.Map()

In [13]:
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [14]:
# Define a Sentinel-2 image collection -> exists over all time and space. An image collection is a set of Earth Engine images. This Image Collection represents every
# Sentinel-2 scene ever collected, all over the Earth

In [15]:
ic = ee.ImageCollection('COPERNICUS/S2_SR')

In [16]:
# Filter by both a region of interest (draw on map) and a time period

In [17]:
start_datetime = '2022-01-01'
end_datetime = '2023-01-01'

In [18]:
ic = ic.filterDate(start_datetime, end_datetime)

In [20]:
roi = m.draw_last_feature.geometry()

In [21]:
# Does not do a CLIP operation to the drawn geometry -> any scenes that intersect this ROI will be preserved, looks through the image collection for
# images containing that rectangle in the date time frame
ic = ic.filterBounds(roi)

In [22]:
# Look at the size of the image collection, this tells us the total number of images we're working with
# remember that you need to stick in a getInfo() to get information
ic.size().getInfo()

7371

In [ ]:
# A mosaic throws all the imagery onto the map

In [23]:
mosaic = ic.mosaic()

In [24]:
m.addLayer(mosaic, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "Mosaic")

In [25]:
# Compute a mean composite -> takes a pixel-wsie mean across the entire timestack

In [26]:
mean = ic.mean()

In [27]:
m.addLayer(mean, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "Mean")

In [ ]:
# The mean image is quite cloudy. This means that NYC is cloudy enough during the year that the clouds significantly affect the mean pixel values
# We can clean this up by filterning by cloud cover. This is a course filter and simply accepts or rejects scenes that go into the composite image calculation
# The metadata parameter for this is CLOUDY_PIXEL_PERCENTAGE

In [46]:
ic_filt = ic.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

In [47]:
ic_filt.size()

In [48]:
mean_filt = ic_filt.mean()
m.addLayer(mean_filt, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "Mean")

In [ ]:
# Calculating NDVI/NDWI
# In order to do this in GEE, we want to define a function that creates these two indices as new bands for every Image,
# which then get added back into our image collection.

In [49]:
def add_ndvi_ndwi(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
  return image.addBands([ndvi, ndwi])

In [50]:
ic_filt = ic_filt.map(add_ndvi_ndwi)

In [52]:
# note that we're not yet down selecting the bands we want to display here
# we're doing a mean across all bands
# GEE won't actually compute this until we select bands to display
mean_filt = ic_filt.mean()

In [53]:
m.addLayer(mean_filt, {'bands': ['NDVI'], 'min': 0, 'max': 0.7, 'palette': ['white', 'green']}, "Mean NDVI")
m.addLayer(mean_filt, {'bands': ['NDWI'], 'min': 0, 'max': 0.7, 'palette': ['white', 'blue']}, "Mean NDWI")